In [1]:
from IPython.display import HTML
HTML('''
<script>
code_show=false;
function code_toggle(){
    if(code_show){$('.prompt, .input, .output_stderr, .output_error').hide();}
    else{$('.input, .prompt, .output_stderr, .output_error').show();}
    code_show=!code_show;
}
function initialize(){
    var output=$('.output_subarea.output_text.output_stream.output_stdout');
    $.merge(output,$('.output_subarea.output_text.output_result'));
    for(var i=0;i<output.length;i++)for(var j=0;j<output[i].children.length;j++)
        output[i].children[j].style.fontFamily='Palatino';
    code_toggle();
}
$(document).ready(initialize);
</script>
Click <a href="javascript:code_toggle()">here</a> to show/hide codes in this notebook.
''')

# Comparing PDFs
Here, we compare two PDFs or a field of PDFs in a naive way. We show the difference of their moments ($\mu$,$\sigma^2$).

In [2]:
import numpy as np
from matplotlib import pyplot as plt
from ipywidgets import interact
import utils

## 1D

Let $X$ ~ $N(\mu_x, \sigma_x^2)$ and $Y$ ~ $N(\mu_y, \sigma_y^2)$ be two random variables, of a normal distribution. We can compare their moments as followed:
$$
    \mu = \mu_x - \mu_y \\
    \sigma^2 = \sigma_x^2 - \sigma_y^2
$$

In [3]:
def normal_dist(x , mean , var):
    prob_density = 1 / (np.sqrt(2*np.pi)*var) * np.exp(-0.5*((x-mean)/var)**2)
    return prob_density

In [4]:
@interact(mu_x=(0.,1.), var_x=(0.1,1.), mu_y=(0.,1.), var_y=(0.1,1.))
def show_moments(mu_x = 0.5, var_x = 0., mu_y = 1., var_y = 0.6):
    space = np.linspace(-2, 4, 100)

    x = normal_dist(space, mu_x, var_x)
    y = normal_dist(space, mu_y, var_y)

    mu = mu_x - mu_y
    var = var_x - var_y

    # P(mu)
    p_mean_x = normal_dist(mu_x, mu_x, var_x)
    p_mean_y = normal_dist(mu_y, mu_y, var_y)
    # P(sigma)

    plt.figure(figsize=(18,8))
    ax1 = plt.subplot(2,1,1)
    plt.plot(space, x)
    plt.plot(space, y)
    #plt.ylim(0, p_mean_x + 0.5 if p_mean_x >= p_mean_y else p_mean_y + 0.5)
    # mean
    plt.vlines(mu_x, 0, 1.0, ls='dashed', colors='k')
    plt.vlines(mu_y, 0, 1.0, ls='dashed', colors='k')

    mu_pos = mu_x if mu_x <= mu_y else mu_y
    plt.text(mu_pos, 1.05, r'$\mu = {:.2f}$'.format(mu))
    ax1.annotate('', xy=(mu_x, 1.0), xytext=(mu_y, 1.0), arrowprops=dict(arrowstyle="|-|"))

    ax2 = plt.subplot(2,1,2)

    x = normal_dist(space, 0, var_x)
    y = normal_dist(space, 0, var_y)

    plt.plot(space, x)
    plt.plot(space, y)
    plt.xlim(-2, 2)
    plt.ylim(0, 1.7)
    plt.vlines(-np.sqrt(var_x), 0, 1.0, ls='-.', colors='k')
    plt.vlines(np.sqrt(var_x), 0, 1.0, ls='-.', colors='k')
    plt.vlines(-np.sqrt(var_y), 0, 1.5, ls='-.', colors='k')
    plt.vlines(np.sqrt(var_y), 0, 1.5, ls='-.', colors='k')

    plt.text(0, 1.05, r'$\sigma_x = {:.2f}$'.format(np.sqrt(var_x)))
    ax2.annotate('', xy=(-np.sqrt(var_x), 1.0), xytext=(np.sqrt(var_x), 1.0), arrowprops=dict(arrowstyle="|-|"))

    plt.text(0, 1.55, r'$\sigma_y = {:.2f}$'.format(np.sqrt(var_y)))
    ax2.annotate('', xy=(-np.sqrt(var_y), 1.5), xytext=(np.sqrt(var_y), 1.5), arrowprops=dict(arrowstyle="|-|"))

    plt.show()

interactive(children=(FloatSlider(value=0.5, description='mu_x', max=1.0), FloatSlider(value=0.1, description=…

# 2D

In [7]:
width = 10
percent = 5
 
#input_range = 8649.794 # max - min value of sinogram
input_range = 30
var = np.ones((64,64)) * input_range
Sigma_in = utils.build_covariance_matrix(var, function=utils.exponential, width=width)

algs = ['ramp', 'cosine', 'hamming', 'hann', 'shepp-logan']

@interact(algo1=algs, algo2=algs)
def compare(algo1='ramp', algo2='cosine'):
    mu1 = np.load('../../Data/iradon/mu_'+algo1+'.npy')
    df_dx1 = np.load('../../Data/iradon/df_dx_'+algo1+'.npy')
    Sigma1 = df_dx1 @ Sigma_in @ np.transpose(df_dx1)
    variance1 = np.reshape(np.abs(np.diagonal(Sigma1)), mu1.shape)
    
    mu2 = np.load('../../Data/iradon/mu_'+algo2+'.npy')
    df_dx2 = np.load('../../Data/iradon/df_dx_'+algo2+'.npy')
    Sigma2 = df_dx2 @ Sigma_in @ np.transpose(df_dx2)
    variance2 = np.reshape(np.abs(np.diagonal(Sigma2)), mu2.shape)
    
    mu = mu1 - mu2
    variance = variance1 - variance2
    
    plt.figure(figsize=(18,6))
    plt.suptitle(algo1)
    plt.subplot(1,2,1)
    plt.imshow(mu1)
    plt.colorbar()

    plt.subplot(1,2,2)
    plt.imshow(variance1)
    plt.colorbar()
    
    plt.show()
    
    plt.figure(figsize=(18,6))
    plt.suptitle(algo2)
    plt.subplot(1,2,1)
    plt.imshow(mu2)
    plt.colorbar()

    plt.subplot(1,2,2)
    plt.imshow(variance2)
    plt.colorbar()
    
    plt.show()
    
    plt.figure(figsize=(18,6))
    plt.suptitle('difference')
    plt.subplot(1,2,1)
    plt.imshow(mu, cmap='RdBu')
    plt.colorbar()

    plt.subplot(1,2,2)
    plt.imshow(variance, cmap='RdBu')
    plt.colorbar()
    
    plt.show()

interactive(children=(Dropdown(description='algo1', options=('ramp', 'cosine', 'hamming', 'hann', 'shepp-logan…